# 0. Initialisation

In [1]:
// Initialisation chemin des données
val path = "/home/georges/IASDfev20/3_BigData/DarioC/TP/MasterIASD_Flights_Project/data/"
val pathA = "/home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/ONTIME/"
val pathQ = "/home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/QCLCD/"

// Initialisation des variables de date
val date1 = "201311"
val date2 = "201511"
val date3 = "201711"

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.1.30:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1610523683936)
SparkSession available as 'spark'


path: String = /home/georges/IASDfev20/3_BigData/DarioC/TP/MasterIASD_Flights_Project/data/
pathA: String = /home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/ONTIME/
pathQ: String = /home/georges/IASDfev20/3_BigData/DarioC/TP_all/data/QCLCD/
date1: String = 201311
date2: String = 201511
date3: String = 201711


# 1. Liste des aéroports de référence
Les fichiers de vols sont utilisés pour constituer une base de référence des aéroports, via leur code à 3 lettres.

Cette liste (une par année par exemple) permet de délimiter le périmètre des stations météo nécessaires.

In [2]:
// liste des aéroports (sur la base de l'origine et de la destination des vols)
// utilisation de 3 dates nov 2013, 2015 et 2017

val data_airp1 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date1+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp1o = data_airp1.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp1d = data_airp1.withColumnRenamed("dest","CallSign").select("CallSign").distinct

val list_airp1 = data_airp1o.union(data_airp1d).distinct

val data_airp2 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date2+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp2o = data_airp2.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp2d = data_airp2.withColumnRenamed("dest","CallSign").select("CallSign").distinct
val list_airp2 = data_airp2o.union(data_airp2d).distinct

val data_airp3 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date3+"_T_ONTIME_REPORTING.csv")
                      .select("origin","dest")
val data_airp3o = data_airp3.withColumnRenamed("origin","CallSign").select("CallSign").distinct
val data_airp3d = data_airp3.withColumnRenamed("dest","CallSign").select("CallSign").distinct
val list_airp3 = data_airp3o.union(data_airp3d).distinct

val liste_airp = list_airp1.union(list_airp2).union(list_airp3).distinct
                          .orderBy($"CallSign".asc)

println(s"Nb aéroports : ${liste_airp.count()}")

Nb aéroports : 320


data_airp1: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp1o: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp1d: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
list_airp1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp2: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp2o: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp2d: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
list_airp2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string]
data_airp3: org.apache.spark.sql.DataFrame = [origin: string, dest: string]
data_airp3o: org.apache.spark.sql.Dat...


# 2. Préparation des données météo : Weather Observation Table (WOT)

La mise en forme des données météo consiste :

* Etape 1 - Dans un premier temps, à récupérer le lien dans les fichiers stations.txt entre le code aéroport de trois letttres et le Wban d'identification des stations. Cela permet :
    * d'une part de faire limiter le périmètre des stations à la liste des aéroports de référence => Weather Observation Table (OT);
    * d'autre part de croiser les données météo avec celle des vols.
    
* Etape 2 - Dans un second temps, à construire, pour chaque station et chaque jour, le relevé des informations météo pour chaque heure (de 0 à 23h) :
    * les stations/dates conservées sont celles où l'on retrouve un relevé complet sur 24 heures ;
    * les informations météo utilisées (pression, température...) sont de niveau horaire, journalier ou mensuelles et sont conservées pour chaque heure dans un vecteur et indéxées pour les informations de nature qualitative.
    
Remarque : dans l'article, seules des données météo horaires semblent prises en compte.

## Etape 1/2 - Lecture des fichiers QCLCD => Weather Observation File (WOF)
Réduction aux stations correspondant aux aéeroports

### (i) Fichier des stations restreint à la liste de référence des aéroports
A noter qu'il existe des aéroports pour lesquels aucune station n'est trouvée

In [3]:
// novembre 2013

val data_station0 = spark.read.format("csv").option("header", "true").option("delimiter", "|")
                              .load(pathQ+date1+"station.txt")
                              .withColumnRenamed("CallSign", "c_CallSign")

val data_station = data_station0.join(liste_airp,data_station0("c_CallSign") === liste_airp("CallSign"),"inner")
                                .select("CallSign", "WBAN","WMO","Name","State","GroundHeight", "Barometer", "TimeZone")

//println(s"Nb stations : ${data_station.count()}")
//data_station.show(5, false)


// Identification des aéroports sans station
/*
val data_missed_airp = liste_airp.join(data_station0, liste_airp("CallSign")===data_station0("c_CallSign"),"left")
                                 .filter("c_CallSign is null")
                                 .select("CallSign")
println(s"Nb missed airports : ${data_missed_airp.count()}")
data_missed_airp.show(false)
*/

data_station0: org.apache.spark.sql.DataFrame = [WBAN: string, WMO: string ... 13 more fields]
data_station: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 6 more fields]


### (ii) Fichier des relevés mensuels (monthly.txt)

In [4]:
// novembre 2013
val data_monthly = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"monthly.txt")
                             .withColumnRenamed("WBAN","c_WBAN")

println(s"monthly data : ${data_monthly.count()}")

/* Finaliser les données mensuelles à récupérer 

data_monthly.printSchema
data_monthly.select("Wban","YearMonth","AvgMaxTemp","AvgMinTemp","AvgTemp","HeatingDegreeDays",
                    "HDDSeasonToDate","ThunderstormDays","HeavyFogDays").show(false)
*/


monthly data : 1225


data_monthly: org.apache.spark.sql.DataFrame = [c_WBAN: string, YearMonth: string ... 48 more fields]


### (iii) Fichier des relevés quotidiens (daily.txt)

In [5]:
// novembre 2013
val data_daily = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"daily.txt")
                             .withColumnRenamed("WBAN","c_WBAN")
                             .select("c_WBAN","YearMonthDay","Tmax","Tmin","Tavg","DewPoint","WetBulb", "Heat",
                                     "Cool","Sunrise","Sunset","CodeSum","Depth","Water1", "SnowFall",
                                     "StnPressure", "SeaLevel", "ResultSpeed", "ResultDir", "AvgSpeed", "Max5Speed",
                                     "Max5Dir", "Max2Speed", "Max2Dir")
println(s"daily data : ${data_daily.count()}")

daily data : 36750


data_daily: org.apache.spark.sql.DataFrame = [c_WBAN: string, YearMonthDay: string ... 22 more fields]


### (iv) Fichier des relevés horaires (hourly.txt)

In [6]:
// novembre 2013
val data_hourly = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                       .load(pathQ+date1+"hourly.txt")
                       .withColumnRenamed("WBAN","c_WBAN")
                       .withColumn("c_Month",substring($"Date", 1, 6))
                       /*.select("c_WBAN","Date","c_Month","Time","StationType","SkyCondition","Visibility","WeatherType",
                               "DryBulbFarenheit","WetBulbFarenheit","DewPointFarenheit",
                               "WindSpeed","WindDirection","StationPressure","SeaLevelPressure","HourlyPrecip")*/

//print("hourly data : ${data_hourly.count()} ")

data_hourly.select("WetBulbCelsius", "RelativeHumidity","WindDirection","WindSpeed","StationPressure",
                   "SkyCondition", "Visibility", "WeatherType", "RecordType")
           .withColumnRenamed("WetBulbCelsius","T")
           .withColumnRenamed("RelativeHumidity","H")
           .withColumnRenamed("WindDirection", "Wd")
           .withColumnRenamed("WindSpeed", "Ws")
           .withColumnRenamed("StationPressure", "P")
           .withColumnRenamed("SkyCondition", "S")
           .withColumnRenamed("Visibility", "V")
           .withColumnRenamed("WeatherType", "WType")
           .withColumnRenamed("RecordType", "RType")
           .show()
/* Article : 
T : Temperature (WetBulbCelsius), H : Humidity (RelativeHumidity), Wd : wind direction (WindDirection), 
Ws : wind speed (WindSpeed), P : barometric pressure (StationPressure)
S : Sky Condition (SkyCondition), V : visibility (Visibility), D : weather phenomena descriptor (WeatherType, Record Type ?)


Non reprises : HourlyPrecip, SeaLevelPressure
*/
data_hourly.printSchema

+----+---+---+---+-----+--------------------+-----+-----+-----+
|   T|  H| Wd| Ws|    P|                   S|    V|WType|RType|
+----+---+---+---+-----+--------------------+-----+-----+-----+
|-4.0| 78|000|  0|21.52|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.52|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.53|              OVC026|10.00|     |   AA|
|-3.3| 82|140|  3|21.52|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.53|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.53|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.52|              OVC026|10.00|     |   AA|
|-3.6| 85|000|  0|21.52|              OVC026|10.00|     |   AA|
|-2.9| 89|250|  8|21.52|              OVC026|10.00|  -SN|   AA|
|-2.2| 82|260|  8|21.52|BKN026 BKN030 OVC060|10.00|  -SN|   AA|
|-2.2| 82|260|  6|21.53|BKN024 BKN032 OVC060|10.00|  -SN|   AA|
|-2.2| 82|270|  8|21.54|       BKN024 OVC032|10.00|     |   AA|
|-2.2| 82|260|  8|21.54|       BKN024 OV

data_hourly: org.apache.spark.sql.DataFrame = [c_WBAN: string, Date: string ... 43 more fields]


### (v) Fichier des précipitations (precip.txt)

In [ ]:
/* L'information semble être déjà reprise dans la variable HourlyPrecip du fichier hourly.txt

val data_precip = spark.read.format("csv").option("header", "true").option("delimiter", ",")
                             .load(pathQ+date1+"precip.txt")

data_hourly.select("Wban", "Date", "Time", "HourlyPrecip").where($"HourlyPrecip" !== " ")
.join(data_precip.select("Wban","YearMonthDay","Hour","Precipitation"), 
     data_hourly("Wban")===data_precip("Wban") && data_hourly("Date")===data_precip("YearMonthDay"), "inner")
.where($"Precipitation" !== " ")
.show(80,false)
*/

### (vi) => Dataframe Weather Observation File (WOF)

In [7]:
// DATAFRAME DE SYNTHESE regroupant l'ensemble des informations météo pertinentes
val data_meteo_a = data_station.join(data_hourly, data_station("WBAN") === data_hourly("c_WBAN"), "inner")
                               .drop("c_WBAN")
//println(s"meteo_a : ${data_meteo_a.count()}")

val data_meteo_b = data_meteo_a.join(data_daily, data_meteo_a("WBAN") === data_daily("c_WBAN") &&
                                                 data_meteo_a("Date") === data_daily("YearMonthDay"), "left")
                               .drop("c_WBAN")
//println(s"meteo_b : ${data_meteo_b.count()}")

val WOF_201311= data_meteo_b.join(data_monthly, data_meteo_b("WBAN") === data_monthly("c_WBAN") &&
                                                data_meteo_b("c_Month") === data_monthly("yearMonth"), "left")
                            .drop("c_WBAN","c_Month", "YearMonth","YearMonthDay")
print(s"WOF_201311 : ${WOF_201311.count()}")
// WOT_201311.printSchema       

WOF_201311 : 296924

data_meteo_a: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 50 more fields]
data_meteo_b: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 73 more fields]
WOF_201311: org.apache.spark.sql.DataFrame = [CallSign: string, WBAN: string ... 119 more fields]


In [8]:
/* Remarque : présence de couples WBAN/Date dans le fichier hourly mais absents de daily ou de monthly.

A voir si l'on se restreint aux observations pour lesquelles nous disposons à la fois des données horaires,
quotidiennes et mensuelles. Ici le périmètre est celui - plus large des données horaires avec valeurs
manquantes sur les daily ou monthly donc */

// écarts hourly et daily
val data_h = data_meteo_a.select("WBAN","Date")
val data_d = data_daily.withColumnRenamed("c_WBAN", "WBAN")
                               .withColumnRenamed("YearMonthDay","Date")
                               .select("WBAN","Date")

val data_sub = data_h.except(data_d)
print("Nb Wban absents data daily : ")
println(data_sub.count())

// écarts hourly et monthly
val data_h2 = data_meteo_a.select("WBAN","c_Month")
val data_m = data_monthly.withColumnRenamed("c_WBAN", "WBAN")
                               .withColumnRenamed("YearMonth","c_Month")
                               .select("WBAN","c_Month")

val data_sub2 = data_h2.except(data_m)
print("Nb Wban absents data monthly :")
println(data_sub2.count())


Nb Wban absents data daily : 784
Nb Wban absents data monthly :27


data_h: org.apache.spark.sql.DataFrame = [WBAN: string, Date: string]
data_d: org.apache.spark.sql.DataFrame = [WBAN: string, Date: string]
data_sub: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [WBAN: string, Date: string]
data_h2: org.apache.spark.sql.DataFrame = [WBAN: string, c_Month: string]
data_m: org.apache.spark.sql.DataFrame = [WBAN: string, c_Month: string]
data_sub2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [WBAN: string, c_Month: string]


## Etape 2/2 - Mise en forme des données météo => Weather Observation Table (WOT)
Cette étape se déroule en deux temps :
* Ajustement du périmètre des relevés : uniquement les stations et les jours pour lesquels les relevés horaires sont disponibles chaque heure sur 24 heures (0 à 23h).
* Création d'une colonne Weather_h (Array) reprenant les 24 relevés horaires

In [8]:
// Echantillon de travail : on se limite ici aux infos météo sélectionnées

val echan_hourly = WOF_201311.select("CallSign", "State", "Date", "Time",
                                     "GroundHeight",
                                     "WetBulbCelsius","RelativeHumidity","WindDirection","WindSpeed","StationPressure",
                                     "SkyCondition", "Visibility", "WeatherType", "RecordType")
                             .withColumnRenamed("WetBulbCelsius","T")
                             .withColumnRenamed("RelativeHumidity","H")
                             .withColumnRenamed("WindDirection", "Wd")
                             .withColumnRenamed("WindSpeed", "Ws")
                             .withColumnRenamed("StationPressure", "P")
                             .withColumnRenamed("SkyCondition", "S")
                             .withColumnRenamed("Visibility","V")
                             .withColumnRenamed("WeatherType","WType")
                             .withColumnRenamed("RecordType","RType")
                             .withColumnRenamed("GroundHeight","GHeight")

echan_hourly.show(10)
//println(echan_hourly.count())

+--------+-----+--------+----+-------+----+---+---+---+-----+--------------------+-----+-----+-----+
|CallSign|State|    Date|Time|GHeight|   T|  H| Wd| Ws|    P|                   S|    V|WType|RType|
+--------+-----+--------+----+-------+----+---+---+---+-----+--------------------+-----+-----+-----+
|     MEI|   MS|20131101|0056|    294|17.4| 97|280|  7|29.52|SCT010 BKN017 OVC060|10.00|     |   SP|
|     MEI|   MS|20131101|0058|    294|17.6| 93|280|  8|29.52|FEW010 BKN014 OVC060|10.00|     |   AA|
|     MEI|   MS|20131101|0113|    294|17.4| 97|250|  8|29.52|FEW010 BKN015 BKN020|10.00|     |   SP|
|     MEI|   MS|20131101|0123|    294|16.7| 90|270|  5|29.51|FEW012 SCT020 BKN055|10.00|     |   SP|
|     MEI|   MS|20131101|0158|    294|16.5| 93|250|  5|29.51|                 CLR|10.00|     |   AA|
|     MEI|   MS|20131101|0258|    294|15.7| 97|220|  5|29.51|       FEW010 SCT050|10.00|     |   AA|
|     MEI|   MS|20131101|0316|    294|16.1|100|000|  0|29.51|       FEW008 SCT050|10.00|   

echan_hourly: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 12 more fields]


### (i) Limitation du périmètre aux stations/Date avec 24 relevés horaires

En vue du croisement avec les données sur les vols :
* Création de la variable DateH  correspondant à Date en format date

* Création de la variable TimeH correspondant à l'heure pleine la plus proche de l'heure de relevé (Time)

* Limitation aux dates pour lesquelles il existe 24 relevés météo par jour (un par heure). En effet, dans certains cas, il y a des relevés intermédiaires ou il y en a moins que 24. La variable _t_ correspond au numéro de l'heure.

In [9]:
import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{concat, lit, udf}
import org.apache.spark.sql.functions.{hour,minute,second}

import org.apache.spark.sql.expressions.Window

// Fonctions d'agrégation (window)
val w_deltaNb = Window
                    .partitionBy("CallSign","Date","Delta")
                    .orderBy($"Delta" .asc)

val w_deltaRg = Window
                    .partitionBy("CallSign","Date")
                    .orderBy($"Delta" .asc)

val hourly = echan_hourly.withColumn("DateH",to_date(unix_timestamp($"Date","yyyyMMdd").cast("timestamp")))
                         .withColumn("Time00",concat(substring($"Time",1,2),lit("00")))
                         .withColumn("DTime", concat(from_unixtime(unix_timestamp($"Date","yyyyMMdd"),"yyyy-MM-dd"), lit(" "), 
                                                     from_unixtime(unix_timestamp($"Time","HHmm"),"HH:mm:ss")).cast("timestamp"))
                         .withColumn("DTimeH", concat(from_unixtime(unix_timestamp($"Date","yyyyMMdd"),"yyyy-MM-dd"), lit(" "),
                                                      from_unixtime(unix_timestamp($"Time00","HHmm"),"HH:mm:ss")).cast("timestamp"))                  
                         .withColumn("Delta",minute($"DTime")-minute($"DTimeH"))
                         .sort($"CallSign" .asc,$"DateH" .asc, $"Time" .asc)
                         .withColumn("CountDelta",count("Delta") over w_deltaNb )
                         .where($"CountDelta"===24)
                         .drop("Time00","Dtime")

// On ne conserve qu'un relevé horaire de 24 heures lorsqu'il y en a plusieurs sur une même journée

val hourly2 = hourly.withColumn("RgDelta", rank() over w_deltaRg )
                    .where($"RgDelta" ===1)               
                    .withColumn("_t_", hour($"DTimeH"))
                    .drop("Delta","CountDelta","RgDelta")

println(s"Nb stations avec relevé météo sur 24h : ${hourly2.count()}")
/*
hourly2.select("CallSign","Date","DateH","Time","DTimeH","_t_")
       .where($"CallSign"==="DTW").show(25, false)
hourly2.printSchema()
*/

Nb stations avec relevé météo sur 24h : 204600


import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{concat, lit, udf}
import org.apache.spark.sql.functions.{hour, minute, second}
import org.apache.spark.sql.expressions.Window
w_deltaNb: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@47eeb923
w_deltaRg: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@31cf40d6
hourly: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 16 more fields]
hourly2: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 15 more fields]


### (ii) Création de la colonne Weather_h reprenant sous forme d'un Array les 24 relevés météo
WOT_201311 est une table de niveau station / Date en heure pleine (DateH) donnant les infos météo sous la forme d'un Array. 3 colonnes : CallSign, DateH et Weather_h

* Dans un premier temps, la colonne SkyCondition, codifiant les couches nuageuses identifiées, est retraitée.
    
    Une couche nuageuse est caractérisée par un code en 3 lettres (CLR, OVC, SCT, ...) suivi d'un code de 3 chiffres correspondant à la hauteur en centaine de milliers de pieds (015 = 1500 pieds). Au code CLR (Clear) n'est pas associé de hauteur.
    * La colonne SkyCondition est splitée en plusiers colonnes ("SC_1", "SC_2", "SC_3") correspondant aux différentes couches nuageuses mesurées (jusqu'à 3 colonnes pour une station automatique). Le nombre de couches nuageuses est stocké dans une colonne "NumLayer".
    * Chaque colonne correspondant à une couche nuageuse est elle-même splitée en 2 (SC_1_code et SC_1_high) afin de séparer le code de 3 lettres (CLR, OVC, SCT...) et la hauteur de la couche. En cas de ciel dégagé, le code CLR est marqué "000" ainsi que la variable caractérisant la hauteur. Le code de couverture nuageuse est ensuite indexé, par ordre alphabétique décroissant ("SC_1_code_ix).

* Le même procédé est appliqué pour la colonne Weather Type (WType) qui contient 3 catégories d'informations et un niveau d'intensité (cf. code METAR) : une description du temps, des précipitation et de l'obsctruction éventuelle du ciel. 
    * Il peut même exister une information sur la situation aux alentours. A noter que ces informations ne sont pas uniques : par exemple il peut y avoir jusque 3 descriptions du temps, des précipitations ou de l'obstruction du ciel  et nous avons considéré ici la première disponible.
    * nous construisons alors pour chaque relevé météo 4 variables : IN (INtensity), VC (ViCinity : alentours), DR (weather DescRiption), PR (PRecipitation), OB (OBscuration).

* Dans un second temps, les infos météo de type numérique (windspeed, station pressure, NumLayer,SC_1_high... ) sont transformées en Double.

La colonne Weather_h peut être construite à partir de 3 listes de données météo :
* les données de base (base)
* les données de base complétées de SkyCondition
* les données de base, de SkyCondition et de WType

In [21]:
// Splitting of SkyCondition column => skyCond_splitted

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column

val skyCond = hourly2.withColumn("SkyC", split($"S", " "))

// Function to split code part and layer high from SkyCondition observation
def splitCode(DFrame : DataFrame, df_col : Column): DataFrame = {
    val fields = List( (df_col.toString+"_code","CLR"),(df_col.toString+"_high","000") )

    fields.foldLeft( (DFrame,1) ){ 
    (tempdf, field) =>
    val newdf = tempdf._1.withColumn(field._1, when(df_col ==="CLR","000").
                                               otherwise(substring(df_col, tempdf._2,3)))
                         .withColumn(field._1, when(col(field._1) .isNull,"000").
                                               otherwise(col(field._1)))    
    (newdf, tempdf._2 + 3)
  }._1
}

// first split 
val numColsSC = skyCond
                 .withColumn("SkyC_size", size($"SkyC"))
                 .agg(max($"SkyC_size"))
                 .head()
                 .getInt(0)
    
val skyCond_first_split = skyCond.select(col("*") +:
                            (1 until numColsSC + 1).map(i => $"SkyC".getItem(i-1).as(s"S_$i")): _*)
                          .withColumn("NumLayer",size($"SkyC"))
                          .drop($"SkyC")

// second split, colonnes "SC_i", i=1 to numColsSC - utilisation de la fonction splitCode
def listColsSC = for (i <- (1 to numColsSC).toList) yield ("S_"+i)

var resultDF = skyCond_first_split
for (x <- listColsSC) {
    resultDF = splitCode(resultDF,col(x))
}
val skyCond_splitted = resultDF.drop(listColsSC:_*)

println(s"Table skyCond_splitted : ${skyCond_splitted.count()} obs.")
/*skyCond_splitted.select("CallSign","Date","Time","S", "NumLayer","S_1_code", "S_1_high", 
                        "S_2_code", "S_2_high", "S_3_code", "S_3_high").show(10)*/
//skyCond_splitted.printSchema

Table skyCond_splitted : 204600 obs.


import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.Column
skyCond: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 16 more fields]
splitCode: (DFrame: org.apache.spark.sql.DataFrame, df_col: org.apache.spark.sql.Column)org.apache.spark.sql.DataFrame
numColsSC: Int = 3
skyCond_first_split: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 19 more fields]
listColsSC: List[String]
resultDF: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 25 more fields]
skyCond_splitted: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 22 more fields]


In [22]:
// Splitting of Weather Type column => wType_splitted

import org.apache.spark.sql.types.{StringType, IntegerType}
import org.apache.spark.sql.functions.regexp_extract

val wType = skyCond_splitted.withColumn("WType_array", split($"WType", " "))

// first split 
val numColsWT = wType
                 .withColumn("WType_array_size", size($"WType_array"))
                 .agg(max($"WType_array_size"))
                 .head()
                 .getInt(0)
    
val wType_first_split = wType.select(col("*") +:
                            (1 until numColsWT + 1).map(i => $"WType_array".getItem(i-1).as(s"WT_$i")): _*)

// second split
def splitWT(DFrame : DataFrame, df_col : Column): DataFrame = {
    val fields = List(df_col.toString+"_1", df_col.toString+"_2", df_col.toString+"_3")

    fields.foldLeft( (DFrame,1) ){ 
    (tempdf, field) =>
        val newdf = tempdf._1.withColumn(field, substring(df_col,tempdf._2,2))                                               
        (newdf, tempdf._2 + 2)
  }._1
}

def listColsWT = for (i <- (1 to numColsWT).toList) yield ("WT_"+i)
def listD = List("MI","PR","BC","DR","BL","SH","TS","FZ","XX")
def listP = List("RA","DZ","SN","SG","IC","PL","GR","GS","UP")
def listO = List("FG","VA","BR","HZ","DU","FU","SA","PY")

var resultDF = wType_first_split

for (x <- listColsWT) {
    val WT_size = WType_first_split
                   .withColumn("WT_size", length(col(x)))
                   .agg(max($"WT_size"))
                   .head()
                   .getInt(0)
    
    resultDF = resultDF.withColumn(x.toString+"_I",regexp_extract(col(x), "[-+]", 0))
                       .withColumn(x.toString+"_VC", when(substring(col(x),1,2)==="VC",substring(col(x),3,WT_size)).otherwise("00") )
                       .withColumn("new_"+x.toString,when( (col(x.toString+"_I") ==="+" || col(x.toString+"_I") ==="-"),
                                                       substring(col(x),2,WT_size))
                                                     .when(substring(col(x),1,2) === "VC","00")
                                                     .otherwise(col(x)) )

    resultDF = splitWT(resultDF,col("new_"+x.toString))
    
    for (y <- (1 until 4)) {
        resultDF = resultDF
                      .withColumn("I"+x.toString.slice(2,4), when(col(x.toString+"_I")==="+","ST")
                                                    .when(col(x.toString+"_I")==="-","LO")
                                                    .otherwise("00"))
                      .withColumnRenamed(x.toString+"_VC","VC"+x.toString.slice(2,4)) 
                      .withColumn("D"+x.toString.slice(2,4)+"_"+y,when(col("new_"+x.toString+"_"+y).isin(listD:_*),
                                                        col("new_"+x.toString+"_"+y) )
                                                    .otherwise("00"))
                      .withColumn("P"+x.toString.slice(2,4)+"_"+y,when(col("new_"+x.toString+"_"+y).isin(listP:_*),
                                                        col("new_"+x.toString+"_"+y) )
                                                    .otherwise("00"))
                      .withColumn("O"+x.toString.slice(2,4)+"_"+y,when(col("new_"+x.toString+"_"+y).isin(listO:_*),
                                                        col("new_"+x.toString+"_"+y) )
                                                     .otherwise("00"))
    }
}

for (y <- (1 until 4)) {
    resultDF = resultDF
                .withColumn("D_"+y, when(col("D_"+y+"_1") !== "00",col("D_"+y+"_1")).otherwise(when(col("D_"+y+"_2") !== "00",col("D_"+y+"_2")).otherwise(col("D_"+y+"_3"))))
                .withColumn("P_"+y, when(col("P_"+y+"_1") !== "00",col("P_"+y+"_1")).otherwise(when(col("P_"+y+"_2") !== "00",col("P_"+y+"_2")).otherwise(col("P_"+y+"_3"))))
                .withColumn("O_"+y, when(col("O_"+y+"_1") !== "00",col("O_"+y+"_1")).otherwise(when(col("O_"+y+"_2") !== "00",col("O_"+y+"_2")).otherwise(col("O_"+y+"_3"))))
}

// restriction to the first descriptor found
val wType_splitted = resultDF
                        .withColumn("DR", when($"D_1" !=="00",$"D_1").when($"D_2" !== "00", $"D_2").otherwise($"D_3"))
                        .withColumn("PR", when($"P_1" !=="00",$"P_1").when($"P_2" !== "00", $"P_2").otherwise($"P_3"))
                        .withColumn("OB", when($"O_1" !=="00",$"O_1").when($"O_2" !== "00", $"O_2").otherwise($"O_3"))
                        .withColumn("IN", when($"I_1" !=="00",$"I_1").when($"I_2" !== "00", $"I_2").otherwise($"I_3"))
                        .withColumn("VC", when($"VC_1" !=="00",$"VC_1").when($"VC_2" !== "00", $"VC_2").otherwise($"VC_3"))
                        .drop("WT_1","WT_2","WT_3","WT_1_I", "WT_2_I","WT_3_I","new_WT_1","new_WT_2","new_WT_3",
                              "D_1_1", "P_1_1", "O_1_1", "new_WT_1_1", "D_2_1", "P_2_1", "O_2_1", "new_WT_2_1",
                              "D_3_1", "P_3_1", "O_3_1", "new_WT_3_1",
                              "D_1_2", "P_1_2", "O_1_2", "new_WT_1_2", "D_2_2", "P_2_2", "O_2_2", "new_WT_2_2",
                              "D_3_2", "P_3_2", "O_3_2", "new_WT_3_2",
                              "D_1_3", "P_1_3", "O_1_3", "new_WT_1_3", "D_2_3", "P_2_3", "O_2_3", "new_WT_2_3",
                              "D_3_3", "P_3_3", "O_3_3", "new_WT_3_3",
                              "I_1","I_2","I_3","D_1","D_2","D_3","O_1","O_2","O_3","P_1","P_2","P_3",
                              "VC_1","VC_2","VC_3","WType_array")

println(s"Table wType_splitted : ${wType_splitted.count()} obs.")
/*wType_splitted.select("WType","VC","DR","IN","PR","OB")
                  .where($"WType" !==" ")
                  .where($"VC" !== "00" )
                  .show()
*/
//wType_splitted.printSchema

Table wType_splitted : 204600 obs.


import org.apache.spark.sql.types.{StringType, IntegerType}
import org.apache.spark.sql.functions.regexp_extract
wType: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 23 more fields]
numColsWT: Int = 3
wType_first_split: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 26 more fields]
splitWT: (DFrame: org.apache.spark.sql.DataFrame, df_col: org.apache.spark.sql.Column)org.apache.spark.sql.DataFrame
listColsWT: List[String]
listD: List[String]
listP: List[String]
listO: List[String]
resultDF: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 83 more fields]
wType_splitted: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 27 more fields]


In [23]:
// string index the categorical attributes S_i_code, IN, VC, DR, PR, OB => hourly2_indexed

import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.Pipeline

def listColsSC_code = for (i <- (1 to numColsSC).toList) yield ("S_"+i+"_code")
val listColsSCWT = listColsSC_code++List("IN", "VC", "DR", "PR", "OB")
val arrayColsSCWT = listColsSCWT.toArray

val SCindexers = arrayColsSCWT.map
{field => 
    new StringIndexer()
    .setInputCol(field)
    .setStringOrderType("alphabetDesc") 
    .setOutputCol(field+"_x") 
}

val pipeline = new Pipeline()
                  .setStages(SCindexers)
val hourly2_indexed = pipeline.fit(wType_splitted).transform(wType_splitted)

/*hourly2_indexed.select("CallSign","Date","Time","NumLayer","S_1_code_x","S_2_code_x","S_3_code_x",
                       "VC_x","IN_x","DR_x","PR_x","OB_x").show(50)*/

println(s"Table hourly2_indexed : ${hourly2_indexed.count()} obs.")
//hourly2_indexed.printSchema

Table hourly2_indexed : 204600 obs.


import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.Pipeline
listColsSC_code: List[String]
listColsSCWT: List[String] = List(S_1_code, S_2_code, S_3_code, IN, VC, DR, PR, OB)
arrayColsSCWT: Array[String] = Array(S_1_code, S_2_code, S_3_code, IN, VC, DR, PR, OB)
SCindexers: Array[org.apache.spark.ml.feature.StringIndexer] = Array(strIdx_66d3c9d360fe, strIdx_4846ed1ed5c8, strIdx_be4a97f3d60c, strIdx_e35ef3ac5fde, strIdx_f6d124e9c6ca, strIdx_4bb58ef0f801, strIdx_307d9c82398b, strIdx_5fad8473e7e9)
pipeline: org.apache.spark.ml.Pipeline = pipeline_0d82ca9d688f
hourly2_indexed: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 35 more fields]


In [29]:
// Tranformation of numeric measures from string to double => hourly2_double

def listColsSC_high = for (i <- (1 to numColsSC).toList) yield ("S_"+i+"_high")
val listColsOther = List("T","H","Wd","Ws","P","V","NumLayer")
val listColsNum = listColsOther ++ listColsSC_high

val hourly2_double = listColsNum.foldLeft(hourly2_indexed){ 
                     (tempdf, colName) => tempdf.withColumn(colName, col(colName).cast("Double")) }

println(s"Table hourly2_double : ${hourly2_double.count()} obs.")
//hourly2_double.printSchema

Table hourly2_double : 204600 obs.


listColsSC_high: List[String]
listColsOther: List[String] = List(T, H, Wd, Ws, P, V, NumLayer)
listColsNum: List[String] = List(T, H, Wd, Ws, P, V, NumLayer, S_1_high, S_2_high, S_3_high)
hourly2_double: org.apache.spark.sql.DataFrame = [CallSign: string, State: string ... 35 more fields]


In [30]:
hourly2_double.printSchema

root
 |-- CallSign: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- GHeight: string (nullable = true)
 |-- T: double (nullable = true)
 |-- H: double (nullable = true)
 |-- Wd: double (nullable = true)
 |-- Ws: double (nullable = true)
 |-- P: double (nullable = true)
 |-- S: string (nullable = true)
 |-- V: double (nullable = true)
 |-- WType: string (nullable = true)
 |-- RType: string (nullable = true)
 |-- DateH: date (nullable = true)
 |-- DTimeH: timestamp (nullable = true)
 |-- _t_: integer (nullable = true)
 |-- NumLayer: double (nullable = false)
 |-- S_1_code: string (nullable = true)
 |-- S_1_high: double (nullable = true)
 |-- S_2_code: string (nullable = true)
 |-- S_2_high: double (nullable = true)
 |-- S_3_code: string (nullable = true)
 |-- S_3_high: double (nullable = true)
 |-- DR: string (nullable = true)
 |-- PR: string (nullable = true)
 |-- OB: string (nullable = true)
 |-- 

In [31]:
//List of basis weather data
def listColsBase = List("T","H","Wd","Ws","P","V")

//List of weather data linked to skyCondition
def listColsSC_x = for (i <- (1 to numColsSC).toList) yield ("S_"+i+"_code_x")
val listColsSCond = List("Numlayer")++listColsSC_high++listColsSC_x

//List of weather data linked to wType (possibility to restrict column to "VC_x" and "DR_x")
def listColsWT_x = List("IN_x", "VC_x", "DR_x", "PR_x", "OB_x")

// Colonnes retenues
val needed_cols : List[String] = (listColsBase ++ listColsSCond ++ listColsWT_x)


listColsBase: List[String]
listColsSC_x: List[String]
listColsSCond: List[String] = List(Numlayer, S_1_high, S_2_high, S_3_high, SC_1_code_x, SC_2_code_x, SC_3_code_x)
listColsWT_x: List[String]
needed_cols: List[String] = List(T, H, Wd, Ws, P, V, Numlayer, S_1_high, S_2_high, S_3_high, SC_1_code_x, SC_2_code_x, SC_3_code_x, IN_x, VC_x, DR_x, PR_x, OB_x)


In [33]:
// Concatenation of meteo caracteristics in an array "Weather" => hourly3
import org.apache.spark.sql.functions.{col,concat_ws}


//List of basis weather data
def listColsBase = List("T","H","Wd","Ws","P","V")

//List of weather data linked to skyCondition
def listColsSC_x = for (i <- (1 to numColsSC).toList) yield ("S_"+i+"_code_x")
val listColsSCond = List("Numlayer")++listColsSC_high++listColsSC_x

//List of weather data linked to wType (possibility to restrict column to "VC_x" and "DR_x")
def listColsWT_x = List("IN_x", "VC_x", "DR_x", "PR_x", "OB_x")

// Colonnes retenues (to be ajusted if needed)
val needed_cols : List[String] = (listColsBase ++ listColsSCond ++ listColsWT_x)

val hourly3 = hourly2_double.withColumn("Weather",array(needed_cols.map(col): _*))
                            .select("CallSign","State","GHeight","DateH", "Time","DTimeH","_t_","Weather")
                            .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
hourly3.show(25, false)
hourly3.printSchema


// vectorization of weather observation in a unique Weather column : IMPOSSIBLE WITH COLLECT_LIST
/*import org.apache.spark.ml.feature.VectorAssembler

def listSC_code = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_code_ix")
val cols = (listSC_code ++ fields).toArray

val assembler = new VectorAssembler()
  .setInputCols(cols)
  .setOutputCol("Weather")

val hourly3 = assembler.transform(hourly2_double)
                       .select("CallSign","DateH", "Time","TimeH","_t_","Weather")
                       .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
*/

// Concatenation in a struct 
/*import org.apache.spark.sql.functions.struct
import org.apache.spark.sql.Column

def listSC_code = for (i <- (1 to numCols).toList) yield ("SC_"+i+"_code_ix")
val needed_cols : List[String] = (listSC_code ++ fields)

// Varable Weather regroupant, pour chaque heure, les infos météo retenues
val hourly3 = hourly2_double.withColumn("Weather", struct(needed_cols.map(col): _*))
                            .select("CallSign","DateH", "Time","TimeH","_t_","Weather")
                            .sort($"CallSign" .asc,$"DateH" .asc, $"Time")
*/



+--------+-----+-------+----------+----+-------------------+---+------------------------------------------------------------------------------------------------------+
|CallSign|State|GHeight|DateH     |Time|DTimeH             |_t_|Weather                                                                                               |
+--------+-----+-------+----------+----+-------------------+---+------------------------------------------------------------------------------------------------------+
|ABE     |PA   |390    |2013-11-01|0051|2013-11-01 00:00:00|0  |[15.5, 84.0, 200.0, 13.0, 29.3, 10.0, 1.0, 16.0, 0.0, 0.0, 4.0, 5.0, 5.0, 2.0, 3.0, 7.0, 6.0, 5.0]    |
|ABE     |PA   |390    |2013-11-01|0151|2013-11-01 01:00:00|1  |[15.9, 87.0, 200.0, 11.0, 29.27, 5.0, 3.0, 33.0, 47.0, 55.0, 6.0, 4.0, 1.0, 2.0, 3.0, 7.0, 2.0, 4.0]  |
|ABE     |PA   |390    |2013-11-01|0251|2013-11-01 02:00:00|2  |[16.1, 84.0, 200.0, 15.0, 29.23, 10.0, 1.0, 28.0, 0.0, 0.0, 2.0, 5.0, 5.0, 1.0, 3.0, 7.0, 2.0, 5

import org.apache.spark.sql.functions.{col, concat_ws}
listColsBase: List[String]
listColsSC_x: List[String]
listColsSCond: List[String] = List(Numlayer, S_1_high, S_2_high, S_3_high, S_1_code_x, S_2_code_x, S_3_code_x)
listColsWT_x: List[String]
needed_cols: List[String] = List(T, H, Wd, Ws, P, V, Numlayer, S_1_high, S_2_high, S_3_high, S_1_code_x, S_2_code_x, S_3_code_x, IN_x, VC_x, DR_x, PR_x, OB_x)
hourly3: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, State: string ... 6 more fields]


In [34]:
// Concatenation of all 24 hourly weather observations in a array "Weather_h" => WOT_201311
val WOT_201311 = hourly3
  .groupBy($"CallSign",$"DateH")
  .agg(collect_list("Weather").as("Weather_h"))
  .sort($"CallSign" .asc,$"DateH" .asc)

println(s"Table WOT, nb stations / Date : ${WOT_201311.count()}")
WOT_201311.show(1, false) 
WOT_201311.printSchema

Table WOT, nb stations / Date : 8525
+--------+----------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

|ABE     |2013-11-01|[[15.5, 84.0, 200.0, 13.0, 29.3, 10.0, 1.0, 16.0, 0.0, 0.0, 4.0, 5.0, 5.0, 2.0, 3.0, 7.0, 6.0, 5.0], [15.9, 87.0, 200.0, 11.0, 29.27, 5.0, 3.0, 33.0, 47.0, 55.0, 6.0, 4.0, 1.0, 2.0, 3.0, 7.0, 2.0, 4.0], [16.1, 84.0, 200.0, 15.0, 29.23, 10.0, 1.0, 28.0, 0.0, 0.0, 2.0, 5.0, 5.0, 1.0, 3.0, 7.0, 2.0, 5.0], [16.8, 81.0, 210.0, 15.0, 29.19, 10.0, 3.0, 23.0, 33.0, 95.0, 6.0, 4.0, 1.0, 1.0, 3.0, 7.0, 2.0, 5.0], [17.4, 81.0, 200.0, 16.0, 29.15, 10.0, 3.0, 22.0, 50.0, 100.0, 4.0, 4.0, 1.0, 1.0, 3.0, 7.0, 2.0, 5.0], [17.6, 78.0, 200.0, 20.0, 29.13, 10.0, 3.0, 23.0, 29.0, 37.0, 1.0, 4.0, 1.0, 2.0, 3.0, 7.0, 6.0, 5.0], [17.8, 76.0, 210.0, 20.0, 29.12, 10.0, 2.0, 28.0, 34.0, 0.0, 6.0, 1.0, 5.0, 2.0, 3.0, 7.0, 6.0, 5.0], [14.5, 84.0, 250.0, 13.0, 29.19, 10.0, 2.0, 24.0, 55.0, 0.0, 1.0, 1.0, 5.0, 2.0, 3.0, 7.0, 6.0, 5.0], [15.0, 84.0, 220.0, 7.0, 29.19, 10.0, 1.0, 65.0, 0.0, 0.0, 2.0, 5.0, 5.0, 2.0, 3.0, 7.0, 6.0, 5.0], [14.7, 81.0, 240.0, 14.0, 29.2, 10.0, 2.0, 75.0, 100.0, 0.0, 

WOT_201311: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, DateH: date ... 1 more field]


# 3. Préparation des données de vols => Flight Table (FT)
La Table des données de vols contient les informations sur les vols et est limitée aux vols non détournés (diverted) ou annulés (canceled) 

Examen des variables :
* DepTime et ArrTime : heure de départ effective

* CRSDepTimee et CRSArrTime : heures de départ et d'arrivée prévues
    * Computerized reservation system (CRS) departure/arrival time is the scheduled departure/arrival time of the flight,
    * wheels off is the time when the wheels of the aircraftleave the ground at the origin airport, and 
    * wheels on is the time when the wheels of the aircraft touch theground at the destination airport.

* The departure delay of  an  aircraft  is  the  difference  between  the  actual departure  time  and  the  CRS  departure time of the flight. 

* Arrival delay equals actual arrival time minusthe scheduled arrival time.

* "dep_time_blk" : créneau horaire. ex. crs_dep_time=1301 => 1300-1359

* différence entre dep_delay et dep_delay_new ? ici dep_delay peut être négatif et traduire un vol en avance


## Etape 1/2 - Limitation du périmètre des vols
* Limitation aux vols pours lesquels correspond une sation météo au départ ou à l'arrivée
* Limitation aux vols non annulés (canceled) et détournés (diverted)
* Récupération, à partir des données de stations (data_station), de la TimeZone pour les aéroports de départ et d'arrivée et calcul du Time_lag entre aéroports
* Création d'une clé entre les vols au départ et les vols à l'arrivée

In [11]:
// novembre 2013

val data_vols1 = spark.read.format("csv").option("header", "true")
                      .load(pathA+date1+"_T_ONTIME_REPORTING_2.csv")
                      .where( ($"Diverted" !== 1) || ($"Cancelled" != 1) )
                      .filter("Tail_Number is not null")
                      .filter("DepTime is not null")

print("Nb de vols non annulés ou non détournés : ")
println(data_vols1.count())


// Limitation aux vols pour lesquels une station météo est trouvée pour l'aroport de départ et d'arrivée
// Calcul du décalage horaire ("Time_lag") entre le lieu de départ et d'arrivée
val data_vols2  = data_vols1.join(data_station.select("CallSign","TimeZone"), data_vols1("Origin") === data_station("CallSign"),"inner")
                           .drop("CallSign")
                           .withColumnRenamed("TimeZone", "dep_TZone")
                           .join(data_station.select("CallSign","TimeZone"), data_vols1("Dest") === data_station("CallSign"),"inner")
                           .drop("CallSign")
                           .withColumnRenamed("TimeZone", "arr_TZone")
                           .withColumn("Time_lag",$"arr_TZone"-$"dep_TZone")
                           .withColumn("Cle", concat($"FlightDate", $"Tail_Number",$"Origin",$"DepTime",$"Dest",$"ArrTime"))

println(s"Nb de vols non annulés ou non détournés avec station météo : ${data_vols2.count()}")
data_vols2.select("cle","FlightDate","Origin", "Tail_Number", "DepTime",
                                   "Dest","ArrTime","ActualElapsedTime", "Time_lag").show(5)

// Table des vols
val FT_201311 = data_vols2.select("cle","FlightDate","Origin","Tail_Number", "Dest","DepTime","CRSDepTime","DepDelay",
                                  "DepDel15", "DepartureDelayGroups","ArrTime","CRSArrTime","ArrDelay","ArrDel15",
                                  "ActualElapsedTime","Distance","Time_lag",
                                  "WeatherDelay","NASDelay","SecurityDelay","CarrierDelay","LateAircraftDelay")                  

Nb de vols non annulés ou non détournés : 498420
Nb de vols non annulés ou non détournés avec station météo : 497177
+--------------------+----------+------+-----------+-------+----+-------+-----------------+--------+
|                 cle|FlightDate|Origin|Tail_Number|DepTime|Dest|ArrTime|ActualElapsedTime|Time_lag|
+--------------------+----------+------+-----------+-------+----+-------+-----------------+--------+
|2013-11-01N8747BD...|2013-11-01|   DTW|     N8747B|   2126| BGM|   2238|            72.00|     0.0|
|2013-11-03N8883ED...|2013-11-03|   DTW|     N8883E|   2136| BGM|   2251|            75.00|     0.0|
|2013-11-04N8709AD...|2013-11-04|   DTW|     N8709A|   2145| BGM|   2259|            74.00|     0.0|
|2013-11-05N8918BD...|2013-11-05|   DTW|     N8918B|   2136| BGM|   2245|            69.00|     0.0|
|2013-11-06N8914AD...|2013-11-06|   DTW|     N8914A|   2133| BGM|   2242|            69.00|     0.0|
+--------------------+----------+------+-----------+-------+----+-------+--

data_vols1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Year: string, Quarter: string ... 108 more fields]
data_vols2: org.apache.spark.sql.DataFrame = [Year: string, Quarter: string ... 112 more fields]
FT_201311: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 20 more fields]


## Etape 2/2 - Préparation de la table des vols avant jointure avec les données météo => flightD et flightA

En vue du croisement avec les données météo sur les douze dernières heures, deux tables sont créés l'une pour les vols au départ (flightD), l'autre pour les vols à l'arrivée (flightA)

### (i) Gestion des dates pour les vols

La gestion des dates et des heures concernent à la fois les informations au départ et les informations à l'arrivée.
La date d'arrivée n'est pas renseignée. Elle est reconstituée (ADate) à partir de la date de départ, de l'heure de départ, de la durée totale du vol (actual_elapsed_time) et du time_lag entre les deux aéroports.
Afin de pouvoir croiser les données de vols avec les relevés météo, des heures de référence pour le départ et l'arrivée (DTimeF et ATimeF) sont créées : elles correspondent à l'heure pleine immédiatement inférieure.

* Création de la variable DDate correspondant à fl_date en format date

* Création de la variable ADate correspondant à la date d'arrivée 

* Création de la variable DDateF (departure) et ADateF (arrival) correspondant à la date de référence (heure pleine immédiatement inférieure) pour le rapprochement avec les relevés météo
    * si l'horaire de vol est après 10h, DateF = fl_date ;
    * pour les vols dont une partie des douze dernières heures couvrent le jour précédent, nous créons une nouvelle observation avec DateF au jour précédent (passage de 502507 obs à 706897 obs).

* Création de la variable DTime / ATime correspondant à l'heure de départ et DTimeF / ATimeF correspondant à l'heure pleine la plus proche de l'heure de départ

* La variable _d_ (departure) et _a_ (arrival) correspond au numéro de l'heure de départ et _d0_ / _a0_ au numéro de l'heure la plus petite du jour de départ(DDateF) ou d'arrivée (ADateF) pour laquelle un relevé météo est à récupérer :
    * si l'heure de départ de référence est DTimeF=12:00, pour ce jour : _d_=12 et _d0_=1 et les relevés horaires 1, ..., 12 seront nécessaires (idem pour _a_ et _a0_);
    * si l'heure de départ de référence est DTimeF=08:00, pour ce jour : le numéro de l'heure de départ est _d_ = 8 et _d0_=0 et les relevés horaires 0,1,...,8 seront nécessaires (idem pour _a_ et _a0_). A noter que dans ce cas, il sera nécessaire de créer un enregistrement complémentaire avec la date du jour précédent et _d_=23 et _d0_=21 (relevés 21,22,23) seront nécessaires ( voir (ii) ).

In [13]:
import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{concat, lit}
import org.apache.spark.sql.functions.{hour,minute,second}

// gestion des dates pour le départ : DDate, DTime, DDateF, DTimeF, _d_ et _d0_
val data_vols3 = data_vols2.select("cle","FlightDate","Origin", "Tail_Number", "DepTime",
                                   "Dest","ArrTime","ActualElapsedTime", "Time_lag")
                           .withColumn("DDate",to_date(unix_timestamp($"FlightDate","yyyy-MM-dd").cast("timestamp")))
                           .withColumn("DDateF",to_date(unix_timestamp($"FlightDate","yyyy-MM-dd").cast("timestamp")))
                           .withColumn("DTime", concat(from_unixtime(unix_timestamp($"DDate","yyyyMMdd"),"yyyy-MM-dd"), lit(" "), 
                                                     from_unixtime(unix_timestamp($"DepTime","HHmm"),"HH:mm:ss")).cast("timestamp"))
                           .withColumn("Time00",concat(substring($"DepTime",1,2),lit("00")))
                           .withColumn("DTimeF", concat(from_unixtime(unix_timestamp($"DDate","yyyyMMdd"),"yyyy-MM-dd"), lit(" "),
                                                      from_unixtime(unix_timestamp($"Time00","HHmm"),"HH:mm:ss")).cast("timestamp"))                     
                           .withColumn("_d_", hour($"DTimeF"))
                           .withColumn("_d0_", when($"_d_" > 11, $"_d_"-11).otherwise(0))
                           .withColumn("_ddb_",when($"_d_" <11, 23))            // renseigné si chevauchement sur deux dates
                           .withColumn("_ddb0_", when($"_d_" <11, $"_d_"+13))   // renseigné si chevauchement sur deux dates           
                           .drop("Time00")

println("Extract de vols au départ : ")
data_vols3.select("DDate","DepTime","DTime","DDateF","DTimeF","_d_", "_d0_", "_ddb_", "_ddb0_").show(5, false)


Extract de vols au départ : 
+----------+-------+-------------------+----------+-------------------+---+----+-----+------+
|DDate     |DepTime|DTime              |DDateF    |DTimeF             |_d_|_d0_|_ddb_|_ddb0_|
+----------+-------+-------------------+----------+-------------------+---+----+-----+------+
|2013-11-28|0959   |2013-11-28 09:59:00|2013-11-28|2013-11-28 09:00:00|9  |0   |23   |22    |
|2013-11-27|0956   |2013-11-27 09:56:00|2013-11-27|2013-11-27 09:00:00|9  |0   |23   |22    |
|2013-11-26|0955   |2013-11-26 09:55:00|2013-11-26|2013-11-26 09:00:00|9  |0   |23   |22    |
|2013-11-25|0956   |2013-11-25 09:56:00|2013-11-25|2013-11-25 09:00:00|9  |0   |23   |22    |
|2013-11-22|0956   |2013-11-22 09:56:00|2013-11-22|2013-11-22 09:00:00|9  |0   |23   |22    |
+----------+-------+-------------------+----------+-------------------+---+----+-----+------+
only showing top 5 rows



import org.apache.spark.sql.functions.unix_timestamp
import org.apache.spark.sql.functions.{to_date, to_timestamp}
import org.apache.spark.sql.functions.{concat, lit}
import org.apache.spark.sql.functions.{hour, minute, second}
data_vols3: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 15 more fields]


In [17]:
// gestion des dates pour l'arrivée : ADate, ATime, ADateF, ATimeF, _a_ et _a0_
val data_vols4 = data_vols3.withColumn("departure_time",from_unixtime(unix_timestamp($"DepTime","HHmm"),"HH:mm:ss"))
                           .withColumn("Time0",from_unixtime(unix_timestamp(lit("2359"),"HHmm"),"HH:mm:ss"))
                           .withColumn("Timeto0",round( (unix_timestamp($"Time0","HH:mm:ss") - 
                                                 unix_timestamp($"departure_time","HH:mm:ss")+1)/60, 2))
                           .withColumn("CorrElapsedTime",$"ActualElapsedTime" + $"Time_lag"*60)
                           .withColumn("ADate", when($"CorrElapsedTime" > $"TimeTo0", date_add(to_date($"DDate","yyyy-MM-dd"),+1))
                                                .otherwise($"DDate"))
                           .withColumn("ADateF",$"ADate")
                           .withColumn("ATime", concat(from_unixtime(unix_timestamp($"ADate","yyyyMMdd"),"yyyy-MM-dd"), lit(" "), 
                                                     from_unixtime(unix_timestamp($"ArrTime","HHmm"),"HH:mm:ss")).cast("timestamp"))
                           .withColumn("Time00",concat(substring($"ArrTime",1,2),lit("00")))
                           .withColumn("ATimeF", concat(from_unixtime(unix_timestamp($"ADate","yyyyMMdd"),"yyyy-MM-dd"), lit(" "),
                                                      from_unixtime(unix_timestamp($"Time00","HHmm"),"HH:mm:ss")).cast("timestamp"))
                           .withColumn("_a_", hour($"ATimeF"))
                           .withColumn("_a0_", when($"_a_" > 11, $"_a_"-11).otherwise(0))
                           .withColumn("_adb_",when($"_a_" <11, 23))
                           .withColumn("_adb0_", when($"_a_" <11, $"_a_"+13))            
                           .drop("Time0","TimeTo0","departure_time")

println("Extract de vols à l'arrivée : ")
data_vols4.select("Dest","DDate","DepTime","ArrTime","ADateF","ATimeF","_a_", "_a0_", "_adb_", "_adb0_").show(10, false)

Extract de vols à l'arrivée : 
+----+----------+-------+-------+----------+-------------------+---+----+-----+------+
|Dest|DDate     |DepTime|ArrTime|ADateF    |ATimeF             |_a_|_a0_|_adb_|_adb0_|
+----+----------+-------+-------+----------+-------------------+---+----+-----+------+
|BGM |2013-11-01|2126   |2238   |2013-11-01|2013-11-01 22:00:00|22 |11  |null |null  |
|BGM |2013-11-03|2136   |2251   |2013-11-03|2013-11-03 22:00:00|22 |11  |null |null  |
|BGM |2013-11-04|2145   |2259   |2013-11-04|2013-11-04 22:00:00|22 |11  |null |null  |
|BGM |2013-11-05|2136   |2245   |2013-11-05|2013-11-05 22:00:00|22 |11  |null |null  |
|BGM |2013-11-06|2133   |2242   |2013-11-06|2013-11-06 22:00:00|22 |11  |null |null  |
|BGM |2013-11-07|2136   |2251   |2013-11-07|2013-11-07 22:00:00|22 |11  |null |null  |
|BGM |2013-11-08|2136   |2303   |2013-11-08|2013-11-08 23:00:00|23 |12  |null |null  |
|BGM |2013-11-10|2136   |2249   |2013-11-10|2013-11-10 22:00:00|22 |11  |null |null  |
|BGM |2013-1

data_vols4: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 25 more fields]


### (ii) Création des enregistrements supplémentaires
Lorque les douzes heures qui précèdent l'heure de départ ou l'heure d'arrivée des vols chevauchent le jour qui précède le départ ou l'arrivée, de nouveaux enregistrements sont créés avec comme DDateF ou ADateF la date du jours qui précède. Ces enregistrements permettront de récupérer les données météo de la veille.
Deux tables sont créées :
* flightD pour les informations de départ ;
* filghtA pour les informations d'arrivée.

In [18]:
// Cas des vols au départ
val vols_dep = data_vols4.select("cle","FlightDate","DDate","DDateF","Origin","Tail_Number","DepTime","DTime",
                                   "DTimeF","_d_","_d0_", "_ddb_", "_ddb0_") 

// Nouvel enregistrement pour les vols dont l'heure de départ est inférieure à 12h (DDateF moins un jour)
val vols_dep_new = vols_dep.where($"_d_"<11)
                           .withColumn("_d0_",$"_ddb0_")
                           .withColumn("_d_",$"_ddb_")
                           .withColumn("DDateF1", date_add(to_date($"DDateF","yyyy-MM-dd"),-1))
                           .withColumn("DDateF",$"DDateF1")
                           .drop("DDateF1")          

val flightD = vols_dep.union(vols_dep_new)
                      .select("cle","FlightDate","DDate","DDateF","Origin","Tail_Number","DepTime","DTime",
                              "DTimeF","_d_","_d0_")

println(s"Table flightD. Nb vols/Date au départ après enrichissement : ${flightD.count()} \n")
println("Extract de vols au départ, aéroport ADK : ")
flightD.where($"Origin"==="ADK").drop("cle").show(5, false)
println("Extract de vols au départ, nouveaux enregistrements : ")
flightD.where($"DDate" !== $"DDateF").drop("cle").show(5, false)

Table flightD. Nb vols/Date au départ après enrichissement : 668074 

Extract de vols au départ, aéroport ADK : 
+----------+----------+----------+------+-----------+-------+-------------------+-------------------+---+----+
|FlightDate|DDate     |DDateF    |Origin|Tail_Number|DepTime|DTime              |DTimeF             |_d_|_d0_|
+----------+----------+----------+------+-----------+-------+-------------------+-------------------+---+----+
|2013-11-03|2013-11-03|2013-11-03|ADK   |N771AS     |1639   |2013-11-03 16:39:00|2013-11-03 16:00:00|16 |5   |
|2013-11-10|2013-11-10|2013-11-10|ADK   |N799AS     |1815   |2013-11-10 18:15:00|2013-11-10 18:00:00|18 |7   |
|2013-11-07|2013-11-07|2013-11-07|ADK   |N771AS     |1828   |2013-11-07 18:28:00|2013-11-07 18:00:00|18 |7   |
|2013-11-21|2013-11-21|2013-11-21|ADK   |N713AS     |1733   |2013-11-21 17:33:00|2013-11-21 17:00:00|17 |6   |
|2013-11-24|2013-11-24|2013-11-24|ADK   |N799AS     |1753   |2013-11-24 17:53:00|2013-11-24 17:00:00|17 |6   |

vols_dep: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 11 more fields]
vols_dep_new: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 11 more fields]
flightD: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 9 more fields]


In [19]:
// Cas des vols à l'arrivée

val vols_arr = data_vols4.select("cle","FlightDate","ATime","ADate","ADateF","ATimeF","Dest","Tail_Number", "ArrTime",
                                 "_a_", "_a0_", "_adb_", "_adb0_")

// Nouvel enregistrement pour les vols dont l'heure d'arrivée est inférieure à 12h (ADateF moins un jour)
val vols_arr_new = vols_arr.where($"_a_"<11)
                           .withColumn("_a0_",$"_adb0_")
                           .withColumn("_a_",$"_adb_")
                           .withColumn("ADateF1", date_add(to_date($"ADateF","yyyy-MM-dd"),-1))
                           .withColumn("ADateF",$"ADateF1")
                           .drop("ADateF1")          

val flightA = vols_arr.union(vols_arr_new)
                      .select("cle","FlightDate","ADate","ADateF","Dest","Tail_Number","ArrTime","ATime",
                              "ATimeF","_a_","_a0_")

println(s"Table flightA. Nb vols/Date à l'arrivée après enrichissement : ${flightA.count()} \n")
println("Extract de vols à l'arrivée, aéroport ADK : ")
flightA.where($"Dest"==="ADK").drop("cle").show(5, false)
println("Extract de vols à l'arrivée, nouveaux enregistrements : ")
flightA.where($"ADate" !== $"ADateF").drop("cle").show(5, false)

Table flightA. Nb vols/Date à l'arrivée après enrichissement : 610650 

Extract de vols à l'arrivée, aéroport ADK : 
+----------+----------+----------+----+-----------+-------+-------------------+-------------------+---+----+
|FlightDate|ADate     |ADateF    |Dest|Tail_Number|ArrTime|ATime              |ATimeF             |_a_|_a0_|
+----------+----------+----------+----+-----------+-------+-------------------+-------------------+---+----+
|2013-11-03|2013-11-03|2013-11-03|ADK |N771AS     |1544   |2013-11-03 15:44:00|2013-11-03 15:00:00|15 |4   |
|2013-11-10|2013-11-10|2013-11-10|ADK |N799AS     |1633   |2013-11-10 16:33:00|2013-11-10 16:00:00|16 |5   |
|2013-11-07|2013-11-07|2013-11-07|ADK |N771AS     |1601   |2013-11-07 16:01:00|2013-11-07 16:00:00|16 |5   |
|2013-11-21|2013-11-21|2013-11-21|ADK |N713AS     |1549   |2013-11-21 15:49:00|2013-11-21 15:00:00|15 |4   |
|2013-11-24|2013-11-24|2013-11-24|ADK |N799AS     |1645   |2013-11-24 16:45:00|2013-11-24 16:00:00|16 |5   |
+----------

vols_arr: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 11 more fields]
vols_arr_new: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 11 more fields]
flightA: org.apache.spark.sql.DataFrame = [cle: string, FlightDate: string ... 9 more fields]


# 4. Table des données de vols enrichie des données météo => JT_201311
Construction du dataframe reprenant les vols avec un colonne Weather_d donnant les relevés météo des douze dernières heures précédant l'heure de départ et Weather_a donnant les relevés météo des douze heures précédant l'heure d'arrivée.

## Etape 1/4 : croisement entre le dataframe des vols et le dataframe reprenant le relevé météo par heure
Les données météo (table WOT_201311) sont rapprochées des données de vol pour le départ (table flightD) et pour l'arrivée (table flightA). Le rapprochement est fait par aéroport et date.

ATTENTION, le croisement correspondant à la première date du jour du mois n'est pas géré ici. Si l'on travaille par mois, il faut préalablement ajouter les relevés météo du dernier jour du mois précédent et du premier jour du mois suivant.

In [19]:
val df_hourly = WOT_201311
df_hourly.printSchema
println(df_hourly.count())

root
 |-- CallSign: string (nullable = true)
 |-- DateH: date (nullable = true)
 |-- Weather_h: array (nullable = true)
 |    |-- element: array (containsNull = false)
 |    |    |-- element: double (containsNull = true)

8525


df_hourly: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallSign: string, DateH: date ... 1 more field]


In [20]:
// Cas des vols au départ

val flightD_H = flightD.join(df_hourly, flightD("Origin") === df_hourly("CallSign") &&
                                         flightD("DDateF") === df_hourly("DateH"), "inner" )
                       .withColumnRenamed("Weather_h","Weather_d")
                       .drop("DateH", "CallSign","Date")
                       .sort($"origin" .asc, $"DDateF" .asc, $"Tail_Number" .asc, $"DTime" .asc)

println(s"Table flightD_H. Nb vols/Date au départ avec relevé météo : ${flightD_H.count()} \n")

//flightD_H.select("fl_date","DDateF","origin", "tail_num","DTime","_d_","_d0_","Weather_d").show(1,false)


Table flightD_H. Nb vols/Date au départ avec relevé météo : 624273 



flightD_H: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, fl_date: string ... 10 more fields]


In [21]:
// Cas des vols à l'arrivée

val flightA_H = flightA.join(df_hourly, flightA("Dest") === df_hourly("CallSign") &&
                                        flightA("ADateF") === df_hourly("DateH"), "inner" )
                       .withColumnRenamed("Weather_h","Weather_a")
                       .drop("DateH", "CallSign","Date")
                       .sort($"Dest" .asc, $"ADateF" .asc, $"Tail_number" .asc, $"ATime" .asc)


println(s"Table flightA_H. Nb vols/Date à l'arrivée avec relevé météo : ${flightA_H.count()} \n")

//flightA_H.select("fl_date","ADateF","dest", "tail_num","ATime","_a_","_a0_","Weather_a").show(1,false)

Table flightA_H. Nb vols/Date à l'arrivée avec relevé météo : 570760 



flightA_H: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, fl_date: string ... 10 more fields]


## Etape 2/4 : élimination des relevés météo qui ne correspondant pas aux douze dernières heures précédant le départ ou l'arrivée du vol
Pour éliminer les relevés météo non nécessaires, les relevés horaires sont repositionnés en ligne à ligne (explode).
Seuls les vols avec 12 relevés horaires disponibles sont retenus (ce qui suppose que les relevés horaires existent pour ce vol sur deux dates adajcentes lorsque les vols partent ou arrivent après 10h du matin).

In [22]:
import org.apache.spark.sql.functions.{explode, flatten}

// cas des vols au départ

val w_hourD = Window
                .partitionBy("origin","DDate","DDateF","tail_num","DTime")
                .orderBy($"DTime" .asc)
val w_obsD = Window
                .partitionBy("origin","DDate","tail_num","DTime")
                .orderBy($"DTime" .asc)

val flightD_H_explode = flightD_H.withColumn("Weather_t",explode($"Weather_d"))
                                 .withColumn("rn", row_number over w_hourD)
                                 .withColumn("_t_", $"rn"-1 )
                                 .where($"_t_">=$"_d0_" && $"_t_"<= $"_d_")  // sélection des relevés horaires
                                 .withColumn("obs", count("DTime") over w_obsD)
                                 .where($"obs" === 12)                       // limitation à 12 relevés horaires
                                 .select("cle","Origin","DepTime","FlightDate","DDate","DDateF","Tail_Number","DTime","_d_","_d0_","_t_","obs","Weather_t")
                                 .sort($"Origin" .asc, $"FlightDate" .asc, $"Tail_Number" .asc, $"DTime" .asc, $"DDateF" .asc, $"_t_" .asc)

println(s"Table flightD_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : ${flightD_H_explode.count()} \n")

println("Exemple de vol au départ : \n")
flightD_H_explode.select("Origin","DepTime","FlightDate","_d_","_d0_","_t_","obs","Weather_t")
                 .where($"Origin"==="ABE" && $"DepTime" === "1140" && $"Tail_Number"==="N29906")
                 .show(24)

Table flightD_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : 5535744 

Exemple de vol au départ : 

+------+--------+----------+---+----+---+---+--------------------+
|origin|dep_time|   fl_date|_d_|_d0_|_t_|obs|           Weather_t|
+------+--------+----------+---+----+---+---+--------------------+
|   ABE|    1140|2013-11-01| 11|   0|  0| 12|[4.0, 5.0, 5.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  1| 12|[6.0, 4.0, 1.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  2| 12|[2.0, 5.0, 5.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  3| 12|[6.0, 4.0, 1.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  4| 12|[4.0, 4.0, 1.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  5| 12|[1.0, 4.0, 1.0, 2...|
|   ABE|    1140|2013-11-01| 11|   0|  6| 12|[6.0, 1.0, 5.0, 2...|
|   ABE|    1140|2013-11-01| 11|   0|  7| 12|[1.0, 1.0, 5.0, 1...|
|   ABE|    1140|2013-11-01| 11|   0|  8| 12|[2.0, 5.0, 5.0, 7...|
|   ABE|    1140|2013-11-01| 11|   0|  9| 12|[4.0, 1.0, 5.0, 1...|
|   ABE|    11

import org.apache.spark.sql.functions.{explode, flatten}
w_hourD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2eda0e7d
w_obsD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3d98249b
flightD_H_explode: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, origin: string ... 11 more fields]


In [23]:
// cas des vols à l'arrivée

val w_hourA = Window
                .partitionBy("Dest","ADate","ADateF","Tail_Number","ATime")
                .orderBy($"ATime" .asc)
val w_obsA = Window
                .partitionBy("Dest","ADate","Tail_Number","ATime")
                .orderBy($"ATime" .asc)

val flightA_H_explode = flightA_H.withColumn("Weather_t",explode($"Weather_a"))
                                 .withColumn("rn", row_number over w_hourA)
                                 .withColumn("_t_", $"rn"-1 )
                                 .where($"_t_">=$"_a0_" && $"_t_"<= $"_a_")
                                 .withColumn("obs", count("ATime") over w_obsA)
                                 .where($"obs"===12)
                                 .select("cle","Dest","ArrTime","FlightDate","ADate","ADateF","Tail_Number","ATime","_a_","_a0_","_t_","obs","Weather_t")
                                 .sort($"Dest" .asc, $"FlightDate" .asc, $"Tail_Number" .asc, $"ATime" .asc, $"ADateF" .asc, $"_t_" .asc)

println(s"Table flightA_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : ${flightA_H_explode.count()} \n")

println("Exemple de vol à l'arrivée : \n")
flightA_H_explode.select("Dest","ArrTime","FlightDate","ADateF","Tail_Number","ATime","_a_","_a0_","_t_","obs","Weather_t")
                 .where($"Dest"==="ABQ" && $"ArrTime" === "0806" && $"Tail_Number"==="N607SW")
                 .show(25)

Table flightA_H_explode. Nb vols/Dates au départ avec 12 relevés horaires : 5557104 

Exemple de vol à l'arrivée : 

+----+--------+----------+----------+--------+-------------------+---+----+---+---+--------------------+
|dest|arr_time|   fl_date|    ADateF|tail_num|              ATime|_a_|_a0_|_t_|obs|           Weather_t|
+----+--------+----------+----------+--------+-------------------+---+----+---+---+--------------------+
| ABQ|    0806|2013-11-02|2013-11-01|  N607SW|2013-11-02 08:06:00| 23|  21| 21| 12|[7.0, 5.0, 5.0, 9...|
| ABQ|    0806|2013-11-02|2013-11-01|  N607SW|2013-11-02 08:06:00| 23|  21| 22| 12|[7.0, 5.0, 5.0, 7...|
| ABQ|    0806|2013-11-02|2013-11-01|  N607SW|2013-11-02 08:06:00| 23|  21| 23| 12|[7.0, 5.0, 5.0, 6...|
| ABQ|    0806|2013-11-02|2013-11-02|  N607SW|2013-11-02 08:06:00|  8|   0|  0| 12|[7.0, 5.0, 5.0, 5...|
| ABQ|    0806|2013-11-02|2013-11-02|  N607SW|2013-11-02 08:06:00|  8|   0|  1| 12|[7.0, 5.0, 5.0, 5...|
| ABQ|    0806|2013-11-02|2013-11-02|  N607

w_hourA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@4dcffa40
w_obsA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@70fdde03
flightA_H_explode: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [cle: string, dest: string ... 11 more fields]


## Etape 3/4 : vols avec leurs 12 relevés météo
On reconstitue pour chaque vol la colonne "Weather_d" (départ) et "Weather_a" (arrivée) regroupant les 12 derniers relevés météo disponibles

In [24]:
// Vols au départ

val w_weatherD = Window
                .partitionBy("Origin","FlightDate","Tail_Number")
                .orderBy($"DepTime" .asc)

val w_rnD = Window
           .partitionBy("origin","FlightDate","Tail_Number","DepTime")
           .orderBy($"DepTime" .asc)

val flightD_H12 = flightD_H_explode.withColumn("Weather_d", collect_list("Weather_t") over w_weatherD)
                                   .withColumn("rn", row_number over w_rnD )
                                   .where($"rn"===12)
                                   .select("cle","Weather_d")

println(s"Nb vols/Date après récupération 12 dernières heures infos météo : ${flightD_H12.count()} \n")
      
flightD_H12.show(1, false)

Nb vols/Date après récupération 12 dernières heures infos météo : 461312 

+------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|cle                           |Weather_d                       

w_weatherD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@6bf5a273
w_rnD: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2866e37
flightD_H12: org.apache.spark.sql.DataFrame = [cle: string, Weather_d: array<array<double>>]


In [25]:
// Vols à l'arrivée

val w_weatherA = Window
                .partitionBy("Dest","FlightDate","Tail_Number")
                .orderBy($"ArrTime" .asc)

val w_rnA = Window
           .partitionBy("Dest","FlightDate","Tail_Number","ArrTime")
           .orderBy($"ArrTime" .asc)

val flightA_H12 = flightA_H_explode.withColumn("Weather_a", collect_list("Weather_t") over w_weatherA)
                                   .withColumn("rn", row_number over w_rnA )
                                   .where($"rn"===12)
                                   .withColumnRenamed("cle","c_cle")
                                   .select("c_cle","Weather_a")

println(s"Nb vols/Date après récupération 12 dernières heures infos météo : ${flightA_H12.count()} \n")
      
flightA_H12.show(1,false)

Nb vols/Date après récupération 12 dernières heures infos météo : 463090 

+------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c_cle                         |Weather_a                                    

w_weatherA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@1ace1677
w_rnA: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@60ae517a
flightA_H12: org.apache.spark.sql.DataFrame = [c_cle: string, Weather_a: array<array<double>>]


## Etape 4/4 : regroupement des vols (départ / arrivée) via la clé => JT_201311
Les tables des données météo associées aux vols au départ et à l'arrivée sont regroupées avec la table des vols FT.

In [26]:
val flightDA_H12 = flightD_H12.join(flightA_H12, flightD_H12("cle")===flightA_H12("c_cle"), "inner")
                              .drop("c_cle")
                              .withColumnRenamed("cle","c_cle")

val JT_201311 = FT_201311.join(flightDA_H12, 
                               FT_201311("cle") === flightDA_H12("c_cle"), "inner")
                              .drop("c_cle")

print(s"Table JT_201311. Nb vols/Date après récupération des données météo : ${JT_201311.count()} \n")
JT_201311.show(1, false)

433248
Table JT_201311. Nb vols/Date après récupération des données météo : 433248 
+------------------------------+----------+------+--------+----+--------+------------+---------+-------------+---------+---------------+--------+------------+---------+-------------+---------+-------------------+--------+--------+-------------+---------+--------------+-------------+-------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

flightDA_H12: org.apache.spark.sql.DataFrame = [c_cle: string, Weather_d: array<array<double>> ... 1 more field]
JT_201311: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 24 more fields]


In [27]:
JT_201311.printSchema

root
 |-- cle: string (nullable = true)
 |-- fl_date: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- tail_num: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- crs_dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- dep_delay_new: string (nullable = true)
 |-- dep_del15: string (nullable = true)
 |-- dep_delay_group: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- crs_arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- arr_delay_new: string (nullable = true)
 |-- arr_del15: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- Time_lag: double (nullable = true)
 |-- weather_delay: string (nullable = true)
 |-- nas_delay: string (nullable = true)
 |-- security_delay: string (nullable = true)
 |-- carrier_delay: string (nullable = true)
 |-- late_aircraft_delay: stri

# 5. Label and target data creation

Evaluation of random forest algorithm is done on a combination of 4 parameters :
* Target dataset (D1, D2, D3, D4)
    * D1 contains only delayed flights due to extreme value weather or NAS, or a combination of them
    * D2 contains delayed flights due to extreme value weather + NAS delay if NAS delay > DT
    * D3 containes delayed flights due to extreme value weather or NAS, even if not exclusively
    * D4 contains all delayed flights
* Delay Threshold (DT)
    We consider 5 delay threshold (DT) in minutes : DT = 15,30,45,60,90)
* Number of hourly weather observations at origin airport (m) : m = 0,1,3,5,7,9,11
* Number of hourly weather observations at destination airport (n) : n = 0,1,3,45,7,9,11

In [28]:
// Fonction construisant les variables cibles (target) selon le seuil de retard donné en paramètre
import org.apache.spark.sql.DataFrame

def produceTarget(DFrame : DataFrame, dt : Int): DataFrame = {
    DFrame.withColumn("target_4_"+dt.toString, when(($"arr_delay" >= dt) && ($"arr_delay" !== $"late_aircraft_delay"), 1).otherwise(0))
                            .withColumn("target_3_"+dt.toString, when((col("target_4_"+dt.toString) === 1) && (($"weather_delay" > 0) || ($"nas_delay" > 0)), 1).otherwise(0))
                            .withColumn("target_2_"+dt.toString, when((col("target_4_"+dt.toString) === 1) && (($"weather_delay" > 0) || ($"nas_delay"> dt)), 1).otherwise(0))
                            .withColumn("target_1_"+dt.toString, when((col("target_4_"+dt.toString) === 1) && (($"weather_delay" > 0) || ($"nas_delay" > 0)) && 
                                                             ($"security_delay" === 0) && ($"carrier_delay" === 0), 1).otherwise(0))
}

// Table JT enrichie des indicateurs de retard

val DT = Array(15,30, 45, 60, 90)

var resultDF = JT_201311
for (i <- 0 to (DT.size-1)) {
    resultDF = produceTarget(resultDF,DT(i))
}
val FT_target_201311 = resultDF


/*FT_target_201311.where($"target_4_30" === 1)
                .withColumnRenamed("late_aircraft_delay","l_a_d")
                .withColumnRenamed("security_delay","s_d")
                .withColumnRenamed("weather_delay","w_d")
                .where($"target_2_30"=== 1)
                .select("arr_delay",
                        "w_d","nas_delay","s_d","carrier_delay","l_a_d",
                        "target_1_30", "target_2_30", "target_3_30", "target_4_30"
                             ).show(10)*/

import org.apache.spark.sql.DataFrame
produceTarget: (DFrame: org.apache.spark.sql.DataFrame, dt: Int)org.apache.spark.sql.DataFrame
DT: Array[Int] = Array(15, 30, 45, 60, 90)
resultDF: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 44 more fields]
FT_target_201311: org.apache.spark.sql.DataFrame = [cle: string, fl_date: string ... 44 more fields]


# 6. Application modèle d'apprentissage supervisé - Regression Tree

In [29]:
FT_target_201311.printSchema

root
 |-- cle: string (nullable = true)
 |-- fl_date: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- tail_num: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- crs_dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- dep_delay_new: string (nullable = true)
 |-- dep_del15: string (nullable = true)
 |-- dep_delay_group: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- crs_arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- arr_delay_new: string (nullable = true)
 |-- arr_del15: string (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- Time_lag: double (nullable = true)
 |-- weather_delay: string (nullable = true)
 |-- nas_delay: string (nullable = true)
 |-- security_delay: string (nullable = true)
 |-- carrier_delay: string (nullable = true)
 |-- late_aircraft_delay: stri

In [ ]:
// Threshold : 15 min
// target : target_1
// m = 3
// n = 1

In [ ]:
// selection of label and features
import org.apache.spark.ml.linalg._

val convertUDF = udf((array : Seq[Double]) => {
  Vectors.dense(array.toArray)
})

val m_d = 1
val n_a = 1
def Cols_d = for (i <- (0 to m_d-1).toList) yield ("W_d_"+i)
def Cols_a = for (i <- (0 to n_a-1).toList) yield ("W_a_"+i)
def listCols = Cols_d ++ Cols_a

val FT_201311_model = FT_target_201311.select(col("target_1_15") +: col("Weather_a") +:
                            (0 until m_d).map(i => $"Weather_d".getItem(11-i).as(s"W_d_$i")): _*)
                                      .select(col("*") +:
                            (0 until n_a).map(i => $"Weather_a".getItem(11-i).as(s"W_a_$i")): _* )
                                      .drop("Weather_a")
                                      .withColumn("label", col("target_1_15").cast("Double"))
                                      .withColumn("Weather",concat(listCols.map(c => col(c)): _*))
                                      .withColumn("features", convertUDF($"Weather"))
                                      .select("label","features")

// sampling
val Array(trainingData, testData) = FT_201311_model.randomSplit(Array(0.7, 0.3))

println(s"Taille de trainingData : ${trainingData.count()} \n")

In [75]:
// decision tree classifier
import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator

val dt = new DecisionTreeClassifier()
  .setLabelCol("label")
  .setFeaturesCol("features")
  /*.setImpurity("entropy").setMaxBins(maxCat)*/

// train model
val dtModel = dt.fit(trainingData)

val treeModel = dtModel.asInstanceOf[DecisionTreeClassificationModel]
//println(s"Learned classification tree model:\n ${treeModel.toDebugString}")

// make predictions
val predictions = dtModel.transform(testData)

// Select (prediction, true label) and compute test error.
import org.apache.spark.ml.evaluation._

val evaluator = new MulticlassClassificationEvaluator()
  .setLabelCol("label")
  .setPredictionCol("prediction")
  .setMetricName("accuracy")
val accuracy = evaluator.evaluate(predictions)
println(s"Test Error = ${(1.0 - accuracy)}")

// feature importance
model.featureImportances


Test Error = 0.050641556926666054


import org.apache.spark.ml.classification.DecisionTreeClassificationModel
import org.apache.spark.ml.classification.DecisionTreeClassifier
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
dt: org.apache.spark.ml.classification.DecisionTreeClassifier = dtc_32ee11bc60e0
dtModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_32ee11bc60e0, depth=5, numNodes=15, numClasses=2, numFeatures=48
treeModel: org.apache.spark.ml.classification.DecisionTreeClassificationModel = DecisionTreeClassificationModel: uid=dtc_32ee11bc60e0, depth=5, numNodes=15, numClasses=2, numFeatures=48
predictions: org.apache.spark.sql.DataFrame = [label: double, features: vector ... 3 more fields]
import org.apache.spark.ml.evaluation...
